In [1]:
# Dataset management packages
from spivutils.synthetic_datasets.spid import load_data
from spivutils.batch_generators.keras_generator import batch_data
from spivutils.common_tools.operations import normalization, vectoraddition, thresholding

# General purpose packages
import numpy as np

# Model packages
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Conv3D, MaxPooling3D, MaxPooling2D,ConvLSTM2D,Flatten, Dense, Reshape, Activation, LSTM, TimeDistributed

In [2]:
# Force CPU usage
tf.config.set_visible_devices([], 'GPU')

In [3]:
# Data importing
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data()


 Checking files ...

 train_x.npy  Found!

 train_y.npy  Found!

 valid_x.npy  Found!

 valid_y.npy  Found!

 test_x.npy  Found!

 test_y.npy  Found!

 Importing data ...

 Data loading successful!


In [4]:
batch_size = 10
chunk_size = 100

# Collect a chunk of the dataset
train_x_chunk = train_x[0:chunk_size]
train_y_chunk = train_y[0:chunk_size]
valid_x_chunk = valid_x[0:chunk_size]
valid_y_chunk = valid_y[0:chunk_size]

# Load data in batches to avoid memory overload
train_batch = batch_data(train_x_chunk, train_y_chunk, batch_size)
valid_batch = batch_data(valid_x_chunk, valid_y_chunk, batch_size)

In [5]:
# Add pre-processing operations
train_batch.add_x_preprocessing_operation(thresholding)
train_batch.add_x_preprocessing_operation(normalization)
train_batch.add_y_preprocessing_operation(vectoraddition)

In [8]:
input_shape = train_batch[0][0][0,].shape
output_shape = train_batch[0][1][0,].shape
input_shape_time = (2,) + input_shape[2:]

# Model hyperparameters
filters = 64
kernel_size = (10,10)
pool_size = (5 ,5)
units = 64

In [9]:
model = Sequential()
model.add(ConvLSTM2D(filters=filters, kernel_size=kernel_size, padding='same', input_shape=input_shape, return_sequences=False))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(units))
model.add(Activation('relu'))
model.add(Dense(np.prod(output_shape)))
model.add(Activation('linear'))
model.add(Reshape(output_shape))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.build((None,) + input_shape)

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_1 (ConvLSTM2D)  (None, 630, 665, 64)      1664256   
                                                                 
 activation_1 (Activation)   (None, 630, 665, 64)      0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 126, 133, 64)     0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1072512)           0         
                                                                 
 dense (Dense)               (None, 64)                68640832  
                                                                 
 activation_2 (Activation)   (None, 64)                0         
                                                      

In [11]:
model.fit(train_batch, validation_data = valid_batch, epochs = 1)

: 

: 